In [ ]:
import os
import h5py
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import rasterio
from rasterio.crs import CRS
from rasterio.enums import Resampling
import rioxarray
import asf_search as asf
from shapely.geometry import Polygon

from utils.compare import make_gif, assign_crs, plot_tifs, plot_difference_maps, reproject_match_tifs, plot_array

sns.set_theme()
%matplotlib inline

In [ ]:
# data from https://deant-data-public-dev.s3.ap-southeast-2.amazonaws.com/index.html?prefix=static-layers/20231224T032111/
rtc_VV_tif = 'data/cslc/OPERA_L2_RTC-S1_T094-200132-IW3_20231224T032111Z_20231224T110416Z_S1A_30_v1.0_VV.tif'
rtc_looks = 'data/cslc/OPERA_L2_RTC-S1-STATIC_T094-200132-IW3_20140403_S1A_30_v1.0_number_of_looks.tif'
rtc_gamma2beta = 'data/cslc/OPERA_L2_RTC-S1-STATIC_T094-200132-IW3_20140403_S1A_30_v1.0_rtc_anf_gamma0_to_beta0.tif'
cslc_vv_h5 = 'data/cslc/OPERA_L2_CSLC-S1_T094-200132-IW3_20231224T032111Z_20231225T081521Z_S1A_VV_v1.0.h5'
cslc_static = 'data/cslc/OPERA_L2_CSLC-S1-STATIC_T004-006637-IW3_20140403_S1A_v1.0.h5'

In [ ]:
# read rtc product
with rasterio.open(rtc_VV_tif) as src:
    rtc = src.read(1)
print(f'rtc burst shape: {rtc.shape}')

# read GammtoBeta product
with rasterio.open(rtc_gamma2beta) as src:
    gamma2beta = src.read(1)
print(f'gamma to beta burst shape: {gamma2beta.shape}')

# calculate rtc in beta
rtc_beta = rtc*gamma2beta

# read in clsc data
h5_data = h5py.File(cslc_vv_h5,'r')
vv = h5_data['data']['VV'][:]
cslc_backscatter = np.abs(vv.real) ** 2 + np.abs(vv.imag) ** 2
print(f'cslc burst shape: {vv.shape}')

print(cslc_backscatter.shape)
print(vv.shape)
print('keys')
print(list(h5_data))
print('data keys')
print(list(h5_data['data'].keys()))
print('metadata keys')
print(list(h5_data['metadata'].keys()))

In [ ]:
import numpy as np
from scipy.ndimage import zoom
import matplotlib.pyplot as plt
# Define the desired dimensions after downscaling
new_shape = rtc.shape  # (rows, columns)
# Calculate the zoom factors for each dimension
zoom_factors = (new_shape[0] / cslc_backscatter.shape[0], new_shape[1] / cslc_backscatter.shape[1])
# Perform the downsampling using scipy.ndimage.zoom
downscaled_cslc_backscatter = zoom(cslc_backscatter, zoom_factors, order=1, mode='nearest')

In [ ]:
print('cslc_backscatter')
plot_array(
    cslc_backscatter,
    vmin='P10',
    vmax='P90'
)
print('rtc')
plot_array(
    rtc,
    vmin='P10',
    vmax='P90'
)
print('downscaled_cslc_backscatter')
plot_array(
    downscaled_cslc_backscatter,
    vmin='P10',
    vmax='P90'
)
print('rtc_beta')
plot_array(
    rtc_beta,
    vmin='P10',
    vmax='P90'
)
print('divide')
plot_array(
    downscaled_cslc_backscatter/rtc_beta,
    vmin='P10',
    vmax='P90'
)